In [1]:

from fast_llm.engine.checkpoint.config import CheckpointLoadConfig
from fast_llm.models.gpt.model import GPTModel, GPTModelConfig
from fast_llm.engine.distributed.config import DistributedConfig
from fast_llm.engine.checkpoint.config import DistributedCheckpointFormat
from pathlib import Path
from safetensors.torch import load_file
from fast_llm.config import NoAutoValidate
from fast_llm.engine.checkpoint.config import CheckpointLoadMetadataConfig
from fast_llm.models.gpt.config import GPTModelConfig

In [5]:
path_to_checkpoint = "/mnt/checkpoints/fast_llm_exp/slam/checkpoints/pt_runs/slam-5_1b-z1-4096-mbs6/checkpoint/290000"

with NoAutoValidate():
    pretrained_config = CheckpointLoadMetadataConfig(
        path=Path(path_to_checkpoint)
    )   
    metadata = GPTModelConfig.load_metadata(pretrained_config)
    # model_config = GPTModelConfig.from_pretrained(
    #     pretrained=pretrained_config,
    #     metadata=metadata
    # )

Loading metadata from /mnt/checkpoints/fast_llm_exp/slam/checkpoints/pt_runs/slam-5_1b-z1-4096-mbs6/checkpoint/290000/metadata.yaml

------------------------------- Loaded metadata --------------------------------
fast_llm_version: 0.2.0
model: gpt
format: distributed
config:
  base_model:
    transformer:
      normalization:
        type: rms_norm
      num_layers: 28
      hidden_size: 4096
      num_attention_heads: 24
      head_groups: 8
      add_linear_biases: false
      ffn_hidden_size: 8192
      kv_channels: 128
      gated: true
      activation_type: silu
      init_method_std: 0.015625
      init_method_std_qkv: 0.015625
      init_method_std_attn_proj: 0.0020879784524408156
      init_method_std_mlp_1: 0.015625
      init_method_std_mlp_2: 0.0020879784524408156
      mlp_lr_scale:
      - null
      '!!! rotary':
        theta: 1000000.0
        type: llama3
    vocab_size: 131072
    use_position_embeddings: false
    tie_word_embeddings: false
    init_method_std_embe

ValueError: Validation failed for checkpoint metadata. See logs above for details.

In [53]:
pretrained_config

CheckpointLoadMetadataConfig(format=<class 'fast_llm.engine.checkpoint.config.FastLLMCheckpointFormat'>, path=PosixPath('/mnt/checkpoints/fast_llm_exp/slam/checkpoints/pt_runs/slam-5_1b-z1-4096-mbs6/checkpoint/290000'), load_config=<ModelConfigType.architecture: 'architecture'>)

In [44]:
path_to_checkpoint = "/mnt/checkpoints/fast_llm_exp/slam/checkpoints/pt_runs/slam-5_1b-z1-4096-mbs6/checkpoint/290000"
with NoAutoValidate():
    checkpoint_config = CheckpointLoadConfig(
        path=Path(path_to_checkpoint),
        format=DistributedCheckpointFormat,
        optimizer_state=False
    )

In [48]:
with NoAutoValidate():
    metadata = GPTModelConfig.load_metadata(checkpoint_config)
    model_config = GPTModelConfig.from_metadata(
        checkpoint_config,
        metadata
    )


------------------------------- Loaded metadata --------------------------------
fast_llm_version: 0.2.0
model: gpt
format: distributed
config:
  base_model:
    transformer:
      normalization:
        type: rms_norm
      num_layers: 28
      hidden_size: 4096
      num_attention_heads: 24
      head_groups: 8
      add_linear_biases: false
      ffn_hidden_size: 8192
      kv_channels: 128
      gated: true
      activation_type: silu
      init_method_std: 0.015625
      init_method_std_qkv: 0.015625
      init_method_std_attn_proj: 0.0020879784524408156
      init_method_std_mlp_1: 0.015625
      init_method_std_mlp_2: 0.0020879784524408156
      mlp_lr_scale:
      - null
      '!!! rotary':
        theta: 1000000.0
        type: llama3
    vocab_size: 131072
    use_position_embeddings: false
    tie_word_embeddings: false
    init_method_std_embed: 0.015625
    cross_entropy_impl: fused
  multi_stage:
    zero_stage: 1
  distributed:
    world_size: 480
    rank: 0
    local_

ValueError: Validation failed for checkpoint metadata. See logs above for details.

In [9]:
# Create single-GPU configuration
distributed_config = DistributedConfig(
    tensor_parallel=1,
    pipeline_parallel=1,
    sequence_data_parallel=1,
    local_world_size=1,
    world_size=1,
)

# Initialize single-GPU environment
distributed = Distributed(config=distributed_config)

In [10]:
import tempfile
consolidated_dir = Path(tempfile.mkdtemp())

In [12]:

rank_files = sorted(checkpoint_dir.glob("rank_*.safetensors"))
consolidated_state = {}

# Read metadata to understand the sharding
tp = metadata.get('tensor_parallel', 1)
pp = metadata.get('pipeline_parallel', len(rank_files) // tp)

In [28]:
for rank_file in rank_files:
    # Use safetensors to load the file
    rank_state = load_file(rank_file)
    shard_state = rank_state['state_shard']
    print(shard_state.keys())

AttributeError: 'Tensor' object has no attribute 'keys'

In [29]:
model_config = metadata.get('model_config', {})

In [31]:
metadata

{'config': {'base_model': {'cross_entropy_impl': 'fused',
   'init_method_std_embed': 0.015625,
   'tie_word_embeddings': False,
   'transformer': {'activation_type': 'silu',
    'add_linear_biases': False,
    'ffn_hidden_size': 8192,
    'gated': True,
    'head_groups': 8,
    'hidden_size': 4096,
    'init_method_std': 0.015625,
    'init_method_std_attn_proj': 0.0020879784524408156,
    'init_method_std_mlp_1': 0.015625,
    'init_method_std_mlp_2': 0.0020879784524408156,
    'init_method_std_qkv': 0.015625,
    'kv_channels': 128,
    'mlp_lr_scale': [None],
    'normalization': {'type': 'rms_norm'},
    'num_attention_heads': 24,
    'num_layers': 28,
    'rotary': {'theta': 1000000.0, 'type': 'llama3'}},
   'use_position_embeddings': False,
   'vocab_size': 131072},
  'distributed': {'distributed_timeout': 7200.0,
   'local_world_size': 8,
   'rank': 0,
   'seed': 984059,
   'training_dtype': 'bfloat16',
   'world_size': 480},
  'multi_stage': {'zero_stage': 1}},
 'fast_llm_ver